In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from data_prep import Password as P
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

USE_CUDA = False
PRE_GEN_ITERS = 100
ADVERSIVE_ITERS = 500
CRITIC_ITERS = 1

device = torch.device("cuda" if torch.cuda.is_available() and USE_CUDA else "cpu")

In [2]:
# custom weights initialization called on G and D
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight.data, 0, 0.001)

In [3]:
class Discriminator(nn.Module):
    def __init__(self, hidden_size):
        super(Discriminator, self).__init__()

        self.input_size = P.n_letters + 1
        self.hidden_size = hidden_size
        self.output_size = 1
        
        self.i2h = nn.Sequential(*[
            nn.Linear(self.input_size + self.hidden_size, self.hidden_size)
        ])
        self.i2o = nn.Sequential(*[
            nn.Linear(self.input_size + self.hidden_size, self.output_size),
        ])
        
        self.i2h.apply(weights_init)
        self.i2o.apply(weights_init)
        
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size).to(device)
    
    def requiresGrad(self):
        for p in self.parameters():
            p.requires_grad = True
            
    def requiresNoGrad(self):
        for p in self.parameters():
            p.requires_grad = False
            
    def calcGradientPenalty(self, real_data, fake_data, LAMBDA = .5):
        if real_data.size()[0] > fake_data.size()[0]:
            fake_data = P.zeroPadding(fake_data, real_data.size(0))
        elif real_data.size()[0] < fake_data.size()[0]:
            real_data = P.zeroPadding(real_data, fake_data.size(0))
        alpha = torch.rand(1).to(device)
        interpolate = alpha * real_data + (1 - alpha) * fake_data
        interpolate.requires_grad = True

        output = self.discriminate(interpolate)

        grad = torch.ones_like(interpolate).to(device)
        gradients = torch.autograd.grad(
                outputs=output,
                inputs=interpolate,
                grad_outputs=grad,
                create_graph=True,
                retain_graph=True,
                only_inputs=True,
            )[0]
        penalty = (gradients.norm(2, dim=2)[0][0] - 1 ) ** 2 * LAMBDA
        return penalty
    
    def train(self, input_tensor, category):
        hidden = self.initHidden()
        self.zero_grad()
        
        for i in range(input_tensor.size()[0]):
            output, hidden = self(input_tensor[i], hidden)
            
        loss = -output * category
        loss.backward(retain_graph = True)

        return output, loss.item()
    
    def discriminate(self, input_tensor):
        hidden = self.initHidden()
        for i in range(input_tensor.size()[0]):
            output, hidden = self(input_tensor[i], hidden)
        return output

In [4]:
class Generator(nn.Module):
    def __init__(self, hidden_size):
        super(Generator, self).__init__()
        
        self.input_size = P.n_letters + 1
        self.hidden_size = hidden_size
        self.output_size = P.n_letters + 1
        
        self.lstm = nn.LSTM(self.input_size, self.hidden_size)
        self.h2o = nn.Linear(self.hidden_size, self.output_size)
        self.dropout = nn.Dropout(0.1)
        self.hidden = self.initHiddenZeros()

    def forward(self, input):
        output, self.hidden = self.lstm(input.view(1,1,-1), self.hidden)
        output = self.h2o(output)
        output = self.dropout(output)
        output = F.log_softmax(output,dim=2)
        return output

    def initHiddenZeros(self):
        return (torch.zeros(1, 1, self.hidden_size).to(device),
                torch.zeros(1, 1, self.hidden_size).to(device))
    
    def initHiddenRand(self):
        return (torch.rand(1,1, self.hidden_size).to(device), 
                torch.rand(1,1, self.hidden_size).to(device))
    
    def generatePassTensor(self, max_length = 18):
        start_letter = p.passwords_string[random.randint(0,len(p.passwords_string) - 1)][0]
        with torch.no_grad():
            input_tensor = P.passwordToInputTensor(start_letter).to(device)
            self.hidden = self.initHiddenZeros()
            password = start_letter

            for c in range(max_length):
                output = self(input_tensor[0])
                output = output.view(1,-1)
                topv, topi = output.topk(1)
                topi = topi[0][0]
                if topi == P.n_letters - 1:
                    break
                else:
                    letter = P.all_letters[topi]
                    password += letter
                input_tensor = P.passwordToInputTensor(letter).to(device)
        
        return P.passwordToInputTensor(password)
        
    
    def generate_N(self, p, n_generate = 100, max_length = 18):
        generate_list = []

        for i in range(n_generate):
            start_letter = p.passwords_string[random.randint(0,len(p.passwords_string) - 1)][0]
            with torch.no_grad():
                input_tensor = P.passwordToInputTensor(start_letter).to(device)
                self.hidden = self.initHiddenZeros()
                output_password = start_letter

                for c in range(max_length):
                    output = self(input_tensor[0])
                    output = output.view(1,-1)
                    topv, topi = output.topk(1)
                    topi = topi[0][0]
                    if topi == P.n_letters - 1:
                        break
                    else:
                        letter = P.all_letters[topi]
                        output_password += letter
                    input_tensor = P.passwordToInputTensor(letter).to(device)
                    
            generate_list.append(output_password)
            
        return generate_list

    
    def pre_train(self, input_line_tensor, target_line_tensor):
        target_line_tensor.unsqueeze_(-1)
        self.hidden = self.initHiddenZeros()

        criterion = nn.NLLLoss()
        optimizer = torch.optim.SGD(self.parameters(), lr=0.1)

        self.zero_grad()
        loss = torch.tensor(0, dtype = torch.float32, requires_grad = True, device = device)

        for i in range(input_line_tensor.size(0)):
            output = self(input_line_tensor[i])
            output = output.view(1,-1)
            l = criterion(output, target_line_tensor[i])
            loss = loss + l

        loss.backward()
        optimizer.step()

        return output, loss.item() / input_line_tensor.size(0)
    
    
    def train(self, D, p, criterion = nn.NLLLoss(), learning_rate = 0.005, max_length = 18):
        start_letter = p.passwords_string[random.randint(0,len(p.passwords_string) - 1)][0]
        start_tensor = P.passwordToInputTensor(start_letter)
        output_tensor = start_tensor
        self.hidden = self.initHiddenZeros()
       
        output = start_tensor.to(device)
        for c in range(max_length):
            output = self(output[0])
            output_tensor = torch.cat((output_tensor, torch.exp(output)), -3)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == P.n_letters - 1:
                break
            
        output = D.discriminate(output_tensor)
        target_tensor = torch.tensor([1], dtype = torch.long, device = device)
        loss = -output
        #loss = criterion(output, target_tensor)
        loss.backward()
        
        for p in self.parameters():
            p.data.add_(-learning_rate, p.grad.data)

In [5]:
g = Generator(128).to(device)
d = Discriminator(256).to(device)
opt_g = torch.optim.Adam(g.parameters(), lr=0.0002, betas=(0.5, 0.999))
opt_d = torch.optim.Adam(d.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [6]:
p = P()
passwords = p.passIter()

INFO:root:Initializing passwords...
DEBUG:root:Loading from existing json file...
INFO:root:Done initializing passwords.


In [7]:
print_every = 100
loss_trend = []
logging.info("---------- Pre-training generator ----------")
for i in range(PRE_GEN_ITERS):
    pas = next(passwords)
    input_tensor = P.passwordToInputTensor(pas).to(device)
    target_tensor = P.passwordToTargetTensor(pas).to(device)
    output, loss = g.pre_train(input_tensor, target_tensor)
    
    if i % print_every == 0:
        loss_trend.append(loss)
        logging.debug("Iter: "+ str(i)+" Loss: "+str(loss))

INFO:root:---------- Pre-training generator ----------
DEBUG:root:Iter: 0 Loss: 4.385491371154785


In [19]:
logger.setLevel(logging.DEBUG)
logging.info("---------- Adversarial Training ----------\n")

for i in range(1, ADVERSIVE_ITERS + 1):
    logging.debug("----------------- %d / %d -----------------" % (i, ADVERSIVE_ITERS))
    logging.debug("Training discriminator...")
    
    d.requiresGrad()
    g.zero_grad()
    
    correct = 0
    for j in range(CRITIC_ITERS):
        L = 0
        # Genuine
        genuine_password_tensor = P.passwordToInputTensor(next(passwords)).to(device)
        category = 1
        output, loss_real = d.train(genuine_password_tensor, category)
        print("real loss: "+str(loss_real))
        L += loss_real
        if output > 0:
            correct = correct + 1
        # Fake
        fake_password_tensor = g.generatePassTensor().to(device)
        category = -1
        output, loss_fake = d.train(fake_password_tensor, category)
        print("fake loss: "+str(loss_fake))
        L += loss_fake
        if output < 0:
            correct = correct + 1
        # Gradient penalty
        loss_gp = d.calcGradientPenalty(genuine_password_tensor, fake_password_tensor, 1)
        loss_gp.backward(retain_graph=True)
        print("g:"+str(loss_gp.item()))
        L += loss_gp
        
        opt_d.step()
        
    logging.debug("Done. Classification accuracy: " + str(correct / CRITIC_ITERS / 2) + " Loss: " + str(L.item()))    
    
    logging.debug("Training generator...")
    
    d.requiresNoGrad()
    
    g.train(d, p)
    
    logging.debug("Done training generator.\n")
    

INFO:root:---------- Adversarial Training ----------

DEBUG:root:----------------- 1 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.4934585690498352
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 2 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.5321651697158813
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 3 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.3566511273384094
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 4 / 500 -----------------
DEBUG:root:Training discriminator...


real loss: 1.0560625791549683
fake loss: -1.5520808696746826
g:0.00255972589366138
real loss: 0.9930590391159058
fake loss: -1.5620405673980713
g:0.03681632876396179
real loss: 1.5509690046310425
fake loss: -1.2230209112167358
g:0.02870303764939308
real loss: 1.5222121477127075


DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.03203435242176056
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 5 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.4895944595336914
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 6 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.39134904742240906
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 7 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.5030427575111389


fake loss: -1.5542702674865723
g:2.376624797761906e-05
real loss: 1.0234472751617432
fake loss: -1.571798324584961
g:0.05875660106539726
real loss: 1.1757439374923706
fake loss: -1.5728542804718018
g:0.005761292297393084
real loss: 1.0702487230300903
fake loss: -1.5754199028015137
g:0.002128405962139368


DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 8 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.043970704078674316
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 9 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.4410381615161896
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 10 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.40543079376220703
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 11 / 500 -----------------
DEBUG:root:Training discriminator...


...

real loss: 10.338041305541992
fake loss: -10.810619354248047
g:0.14015917479991913
real loss: 10.289726257324219
fake loss: -10.404458999633789
g:0.009391513653099537
real loss: 10.517175674438477
fake loss: -10.856155395507812
g:0.0028329112101346254


DEBUG:root:Done training generator.

DEBUG:root:----------------- 496 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 92.36235046386719
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 497 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 35.55141830444336
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 498 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 209.4132080078125
DEBUG:root:Training generator...


real loss: 10.359780311584473
fake loss: -10.950222969055176
g:92.95279693603516
real loss: 8.530238151550293
fake loss: -9.294565200805664
g:36.31574630737305
real loss: 7.976924419403076
fake loss: -8.547250747680664
g:209.98353576660156


DEBUG:root:Done training generator.

DEBUG:root:----------------- 499 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: 0.3559790253639221
DEBUG:root:Training generator...
DEBUG:root:Done training generator.

DEBUG:root:----------------- 500 / 500 -----------------
DEBUG:root:Training discriminator...
DEBUG:root:Done. Classification accuracy: 0.5 Loss: -0.23035652935504913
DEBUG:root:Training generator...
DEBUG:root:Done training generator.



real loss: 8.572864532470703
fake loss: -9.122283935546875
g:0.905398428440094
real loss: 8.936383247375488
fake loss: -9.402112007141113
g:0.23537223041057587


In [18]:
g.generate_N(p)

['baaaea',
 'Foaae',
 'taaaaae',
 '2aaaa',
 'daaaaa',
 'paaaa',
 'maaaa',
 'caaaa',
 'maaaa',
 'raeaa',
 'faaaa',
 '1aeea',
 'taaaa',
 '2aaaa',
 '0aaaea',
 'maaaa',
 '5aeaa',
 'laaaa',
 '1aaaa',
 'paaae',
 'taaaa',
 'aaaea',
 'aaaaa',
 '1aaaa',
 'laaaa',
 'daaea',
 'laaaa',
 'daaa',
 '*aaaa',
 'iaaaa',
 '9aaee',
 'haaaa',
 'Faeaa',
 'Naaaa',
 '1aaaa',
 'haeea',
 'waaaa',
 'maaaa',
 'maaaa',
 'paaea',
 'jaaaa',
 '8aaaaa',
 'Oaaaa',
 '7aaaa',
 'aaaaa',
 'daaae',
 '8aaaa',
 'laeaa',
 'naaaa',
 'eaaea',
 'vaaaa',
 '1aaaa',
 '9aaaa',
 'uaaaa',
 '0aaaa',
 '2aae',
 'Daaaa',
 '1aaaa',
 'eoaaa',
 'raaaaa',
 'yaaaaae',
 'jaeaa',
 '6aaaaa',
 '2aaaa',
 'raaaa',
 'haaaa',
 '6aeaa',
 '1aaea',
 'taaaa',
 '2aeaa',
 'eaaaa',
 '5oaaa',
 'eaaaaa',
 '0aaaa',
 '1aaae',
 '1aaae',
 'naaaa',
 'haaaa',
 'naaaa',
 'naaea',
 'paaea',
 'maaaa',
 '9aaaa',
 'saaaa',
 'oaaaa',
 '2eaee',
 '0aaaa',
 'iaaaa',
 'caaae',
 'zaaaa',
 'maaaa',
 'saaaa',
 'gaaaa',
 'saaaa',
 'daaea',
 'haaaa',
 'eaaea',
 '0aaaa',
 'taaaa',
 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%matplotlib inline

logger.setLevel(logging.INFO)

plt.figure()
plt.plot(loss_trend)
#plt.savefig('pictures/rnn-rnn-loss.png',dpi=400)
plt.show()

In [ ]:
m = nn.LogSoftmax(dim=1)
loss = nn.NLLLoss()
# input is of size N x C = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = loss(m(input), target)

In [ ]:
import torch
a = torch.ones(3,2)
a.requires_grad = True
b = a * 2
ones = torch.ones_like(a)
g = torch.autograd.grad(
            outputs=b,
            inputs=a,
            grad_outputs=ones,
            create_graph=True,
            retain_graph=True,
            only_inputs=True,
        )

In [ ]:
g

In [ ]:
g[0].view(g[0].size(0), -1).pow(2).sum(1)**(3)

In [38]:
torch.ones(1,1,2).norm(2, dim=2)[0][0]

tensor(True)

In [30]:
g[0]

TypeError: 'Generator' object is not subscriptable

In [ ]:
a = [0,1,2]
a[-1]